In [39]:
import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdf2image import convert_from_path
import PIL
from PIL import Image, ImageFilter

In [13]:
pdfFileObj = open('test.pdf', 'rb') 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
print(pdfReader.numPages) 

1


In [14]:
pageObj = pdfReader.getPage(0) 

In [15]:
print(pageObj.cropBox.getLowerLeft())
print(pageObj.cropBox.getUpperLeft())
print(pageObj.cropBox.getUpperRight())
print(pageObj.cropBox.getLowerRight())

(0, 0)
(0, 792)
(612, 792)
(612, 0)


In [2]:
11*72

792

In [38]:
## Got the first row
## need to turn image into pixels

In [18]:
x = 3 ## Controls the image on the first line
writer = PdfFileWriter()
pageObj.cropBox.setLowerLeft(( 72*(1 + x*1) ,72*11 - 72*2.5))
pageObj.cropBox.setUpperRight((72*(1.45 + x*1.05), 72*11 - 72*2))
writer.addPage(pageObj)

In [19]:
outstream = open('image_crop.pdf','wb')
writer.write(outstream)
outstream.close()

In [27]:
def get_pixel(image, i, j):
    # Inside image bounds?
    width, height = image.size
    if i > width or j > height:
        return None

    # Get Pixel
    pixel = image.getpixel((i, j))
    return pixel

In [28]:
im = Image.open('A.png', 'r')
pix_val = list(im.getdata())
pix_val_flat = [x for sets in pix_val for x in sets]

0

In [36]:
im.size

(180, 180)

In [44]:
def imageprepare(argv):
    
    im = Image.open(argv).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28,28), (255)) ## Create white canvas of 28x28
    
    
    if width > height:
        nheight = int(round((20.0 / width * height), 0))
        if (nheight == 0):
            nheight = 1
            
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight)/ 2),0))
        newImage.paste(img, (4, wtop))
    else:   
        nwidth = int(round((20.0 / height * width), 0))
        if(nwidth == 0):
            nwidth = 1
        img = im.resize((nwidth,20),Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))
        newImage.paste(img, (wleft,4))

    tv = list(newImage.getdata())
    
    tva = [(255 - x)*1.0 / 255.0 for x in tv]
  
    return tva

In [45]:
d = imageprepare('A.png')

In [ ]:
https://towardsdatascience.com/image-classification-in-10-minutes-with-mnist-dataset-54c35b77a38d
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

ImportError: No module named 'numpy.core._multiarray_umath'

SystemError: PyEval_EvalFrameEx returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import